# YOLOv8 Webcam Real-Time Detection

## Install Necessary Libraries

In [1]:

# Install YOLOv8 and OpenCV
!pip install ultralytics opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 24.9 MB/s eta 0:00:00


In [5]:

import cv2
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import numpy as np

# Function to capture webcam feed
def capture_webcam():
    js = Javascript('''
        async function getVideo() {
            const video = document.createElement('video');
            video.style.position = 'fixed';
            video.style.top = 0;
            video.style.left = 0;
            video.style.width = '100%';
            video.style.height = '100%';
            video.style.zIndex = '1000';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            const ctx = canvas.getContext('2d');

            function snapshot() {
                ctx.drawImage(video, 0, 0);
                return canvas.toDataURL('image/jpeg', 0.8);
            }

            window.stream = stream;
            return new Promise(resolve => {
                video.onclick = () => {
                    resolve(snapshot());
                    stream.getTracks().forEach(track => track.stop());
                    video.remove();
                };
            });
        }
    ''')
    display(js)
    data = eval_js('getVideo()')
    img = cv2.imdecode(np.frombuffer(data.split(',')[1].decode('base64'), np.uint8), -1)
    return img

# Capture image from webcam
img = capture_webcam()

# Display captured image
cv2_imshow(img)

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

## Load the YOLOv8 Model

In [6]:

from ultralytics import YOLO

# Load the trained model
model_path = "/content/best.pt"  # Replace with the path to your best.pt file
model = YOLO(model_path)
print(f"Model loaded successfully from: {model_path}")


Model loaded successfully from: /content/best.pt


## Real-Time Detection Using Laptop Camera

In [7]:

import cv2
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function for real-time detection using the webcam
def real_time_detection():
    cap = cv2.VideoCapture(0)  # Open the webcam (0 is the default camera)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Perform inference
        results = model.predict(frame, conf=0.5)  # Adjust confidence threshold as needed

        # Annotate the frame with detection results
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the real-time detection
real_time_detection()


Using device: cpu
Error: Could not open webcam.


### Notes:
- Press **'q'** to exit the webcam detection.
- Replace `best.pt` with the actual path to your trained model if it's not in the current directory.